# 추천 시스템 (Recommender System)

- 추천시스템은 크게 두가지로 구분 가능
  - 컨텐츠 기반 필터링 (content-based filtering)
  - 협업 필터링 (collaborative filtering)
- 두가지를 조합한 hybrid 방식도 가능
- 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
- 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Suprise
- 추천 시스템 개발을 위한 라이브러리
- 다양한 모델과 데이터 제공
- scikit-learn과 유사한 사용 방법


In [1]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 2.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619417 sha256=0a17ba0caf6eb1a950d4736df3c8bd15b61ed41c1ba45c16044608990e1c3cda
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [3]:
data = Dataset.load_builtin("ml-100k", prompt = False)
# (user_id, movie_id, rating, time)
data.raw_ratings[: 10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [4]:
model = SVD()

In [5]:
# 교차검증
cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9358  0.9346  0.9427  0.9349  0.9319  0.9360  0.0036  
MAE (testset)     0.7369  0.7380  0.7436  0.7369  0.7344  0.7380  0.0031  
Fit time          4.75    5.78    5.93    4.99    5.14    5.32    0.46    
Test time         0.25    0.23    0.26    0.26    0.16    0.23    0.04    


{'fit_time': (4.746509075164795,
  5.775949001312256,
  5.927760362625122,
  4.9924561977386475,
  5.143250226974487),
 'test_mae': array([0.73693364, 0.73802901, 0.74359946, 0.73692112, 0.7343537 ]),
 'test_rmse': array([0.93581349, 0.93462836, 0.94266237, 0.93491003, 0.93189712]),
 'test_time': (0.24832677841186523,
  0.2279517650604248,
  0.2550208568572998,
  0.2630784511566162,
  0.16138434410095215)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [6]:
import numpy as np
from surprise import Dataset

- 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
- 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [7]:
data = Dataset.load_builtin("ml-100k", prompt = False)
raw_data = np.array(data.raw_ratings, dtype = int)
raw_data

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116],
       ...,
       [      276,      1090,         1, 874795795],
       [       13,       225,         2, 882399156],
       [       12,       203,         3, 879959583]])

In [8]:
raw_data[ : , 0] -= 1
raw_data[ : , 1] -= 1 

In [9]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])

shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [10]:
# 인접행렬 만들기
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data :
  adj_matrix[user_id][movie_id] = 1.  # 1을 넣어주는 이유는 영화를 봤는지 안봤는지만 알고 싶어서
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [11]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
  if my_id != user_id :   # 나 말고 다른사람들과
    similarity = np.dot(my_vector, user_vector)
    if similarity > best_match :    # 유사도가 best_match보다 높을 경우
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector

# 나와 가장 잘 맞는 user는 275번이며, 유사도는 183이다
print("Best Match : {}, Best Match ID : {}".format(best_match, best_match_id))

Best Match : 183, Best Match ID : 275


In [12]:
# 추천 목록 - 275번 유저의 시청목록
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)) :
  log1, log2 = log
  if log1 < 1. and log2 > 0. :
    recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [13]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []   # 초기 거리를 크게 설정

for user_id, user_vector in enumerate(adj_matrix) :
  if my_id != user_id :   # 나 말고 다른사람들과
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match :    # 유클리디안 거리가 best_match보다 낮을 경우
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector

# 나와 가장 잘 맞는 user는 737번이며, 유클리디안 거리는 14.83이다
print("Best Match : {}, Best Match ID : {}".format(best_match, best_match_id))

Best Match : 14.832396974191326, Best Match ID : 737


In [14]:
# 추천 목록 - 737번 유저의 시청목록
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)) :
  log1, log2 = log
  if log1 < 1. and log2 > 0. :
    recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [15]:
# 코사인 유사도 계산
def compute_cos_similarity(v1, v2) :
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1, v2)
  return dot / (norm1 * norm2)

In [16]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []   # 초기 유사도 설정

for user_id, user_vector in enumerate(adj_matrix) :
  if my_id != user_id :   # 나 말고 다른사람들과
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match :    # 코사인 유사도가 best_match보다 높을 경우
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

# 나와 가장 잘 맞는 user는 915번이며, 코사인 유사도는 0.5279이다
print("Best Match : {}, Best Match ID : {}".format(best_match, best_match_id))

Best Match : 0.5278586163659506, Best Match ID : 915


In [17]:
# 추천 목록 - 915번 유저의 시청목록
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)) :
  log1, log2 = log
  if log1 < 1. and log2 > 0. :
    recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [18]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data :
  adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [19]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []   # 초기 거리를 크게 설정

for user_id, user_vector in enumerate(adj_matrix) :
  if my_id != user_id :   # 나 말고 다른사람들과
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match :    # 유클리디안 거리가 best_match보다 낮을 경우
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector

# 나와 가장 잘 맞는 user는 737번이며, 유클리디안 거리는 55.06이다
print("Best Match : {}, Best Match ID : {}".format(best_match, best_match_id))

Best Match : 55.06359959174482, Best Match ID : 737


In [20]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []   # 초기 유사도 설정

for user_id, user_vector in enumerate(adj_matrix) :
  if my_id != user_id :   # 나 말고 다른사람들과
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match :    # 코사인 유사도가 best_match보다 높을 경우
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

# 나와 가장 잘 맞는 user는 915번이며, 코사인 유사도는 0.5691이다
print("Best Match : {}, Best Match ID : {}".format(best_match, best_match_id))

Best Match : 0.569065731527988, Best Match ID : 915


점수가 추가되니까, 유클리디안 거리와 코사인 유사도가 조금씩 달라졌다.

그런데 우연의 일치일지는 모르겠지만 둘 다 잘 맞는 유저는 바뀌지 않았다.

## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [24]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [25]:
data = Dataset.load_builtin('ml-100k', prompt = False)

- KNN을 사용한 협업필터링

In [26]:
model = KNNBasic()
cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, n_jobs = -1, verbose = True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9818  0.9706  0.9756  0.9838  0.9810  0.9786  0.0048  
MAE (testset)     0.7735  0.7671  0.7713  0.7767  0.7731  0.7723  0.0031  
Fit time          0.25    0.42    0.25    0.37    0.26    0.31    0.07    
Test time         2.95    4.54    4.04    3.52    2.95    3.60    0.62    


{'fit_time': (0.2463381290435791,
  0.41773271560668945,
  0.2492961883544922,
  0.36611151695251465,
  0.256441593170166),
 'test_mae': array([0.77349142, 0.76712306, 0.77131132, 0.77668538, 0.77311321]),
 'test_rmse': array([0.98184399, 0.97061437, 0.97559205, 0.98384099, 0.98102174]),
 'test_time': (2.9451780319213867,
  4.538135051727295,
  4.041172742843628,
  3.5225698947906494,
  2.9451956748962402)}

- NMF를 이용한 협업필터링

In [27]:
model = NMF()
cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, n_jobs = -1, verbose = True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9663  0.9696  0.9535  0.9678  0.9645  0.9644  0.0057  
MAE (testset)     0.7606  0.7650  0.7482  0.7594  0.7577  0.7582  0.0056  
Fit time          6.23    6.58    8.36    6.04    5.08    6.46    1.07    
Test time         0.25    0.24    0.23    0.15    0.15    0.20    0.05    


{'fit_time': (6.226541757583618,
  6.584396123886108,
  8.355381488800049,
  6.0418381690979,
  5.0840160846710205),
 'test_mae': array([0.76058171, 0.76499662, 0.74815275, 0.75944076, 0.75765273]),
 'test_rmse': array([0.96633433, 0.96959249, 0.95353699, 0.96779594, 0.9645112 ]),
 'test_time': (0.24883770942687988,
  0.23676633834838867,
  0.23404312133789062,
  0.14978289604187012,
  0.14689135551452637)}

- SVD를 이용한 협업필터링

In [29]:
model = SVD()
cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, n_jobs = -1, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9318  0.9297  0.9376  0.9416  0.9400  0.9361  0.0046  
MAE (testset)     0.7341  0.7334  0.7384  0.7406  0.7408  0.7375  0.0032  
Fit time          6.32    7.14    7.55    6.34    5.22    6.51    0.80    
Test time         0.28    0.38    0.29    0.17    0.17    0.26    0.08    


{'fit_time': (6.316523313522339,
  7.136928081512451,
  7.550186395645142,
  6.341644048690796,
  5.2177040576934814),
 'test_mae': array([0.73405994, 0.73341666, 0.73836883, 0.74061313, 0.74079238]),
 'test_rmse': array([0.93182841, 0.9297243 , 0.93757502, 0.94158193, 0.94002712]),
 'test_time': (0.27743053436279297,
  0.37557101249694824,
  0.2927072048187256,
  0.1676473617553711,
  0.17466306686401367)}

- SVD++를 이용한 협업필터링

In [28]:
model = SVDpp()
cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, n_jobs = -1, verbose = True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9103  0.9219  0.9300  0.9206  0.9096  0.9185  0.0077  
MAE (testset)     0.7117  0.7228  0.7287  0.7233  0.7124  0.7198  0.0067  
Fit time          334.42  340.10  326.74  334.46  162.52  299.65  68.70   
Test time         6.77    6.14    5.92    6.05    3.05    5.59    1.30    


{'fit_time': (334.41858315467834,
  340.1013717651367,
  326.74077558517456,
  334.4617817401886,
  162.5164897441864),
 'test_mae': array([0.71166925, 0.72280508, 0.72874877, 0.72327327, 0.71243112]),
 'test_rmse': array([0.91025831, 0.92191541, 0.93002524, 0.92061757, 0.90964545]),
 'test_time': (6.765017509460449,
  6.139310836791992,
  5.919731378555298,
  6.054954290390015,
  3.0473744869232178)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [30]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [57]:
data = Dataset.load_builtin('ml-100k', prompt = False)
raw_data = np.array(data.raw_ratings, dtype = int)
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [58]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])

shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [59]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data :
  adj_matrix[user_id][movie_id] = rating

In [60]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [47]:
# 유저, 특이값, 아이템 벡터
U, S, V = randomized_svd(adj_matrix, n_components = 2)
S = np.diag(S)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


In [48]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [49]:
np.matmul(np.matmul(U, S), V)

array([[ 1.76446873e+01, -8.22614770e+01, -2.83543965e+01, ...,
        -4.69924985e+02, -3.37561969e+01, -2.76134569e+02],
       [-1.04165572e+02, -2.85846560e+02,  2.91323233e+02, ...,
         3.45399562e+02,  2.22663748e+01, -7.09996958e+00],
       [-1.43929011e+01, -4.47664573e+01,  4.10996986e+01, ...,
         3.11334141e+01,  1.86740889e+00, -1.21654417e+01],
       ...,
       [ 1.92273119e-04, -7.23550930e-03,  7.09359759e-04, ...,
        -2.50778625e-02, -1.82233207e-03, -1.64621496e-02],
       [ 2.64488810e-02,  3.68861786e-02, -6.82364924e-02, ...,
        -2.00073738e-01, -1.38435200e-02, -7.39478444e-02],
       [ 3.46045501e-02,  4.93118112e-02, -8.94465452e-02, ...,
        -2.58457081e-01, -1.78709733e-02, -9.45184330e-02]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [50]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []   # 초기 유사도 설정

for user_id, user_vector in enumerate(U) :
  if my_id != user_id :   # 나 말고 다른사람들과
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match :    # 코사인 유사도가 best_match보다 높을 경우
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

# 나와 가장 잘 맞는 user는 788번이며, 코사인 유사도는 0.9999이다
print("Best Match : {}, Best Match ID : {}".format(best_match, best_match_id))

Best Match : 0.9999998025256961, Best Match ID : 788


In [52]:
# 추천 목록 - 788번 유저의 시청목록
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])) :
  log1, log2 = log
  if log1 < 1. and log2 > 0. :
    recommend_list.append(i)
print(recommend_list)

[275, 283, 285, 287, 292, 293, 474, 507, 590, 627, 740, 741, 761, 1006, 1007, 1011, 1016, 1160]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [53]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []   # 초기 유사도 설정

for user_id, user_vector in enumerate(V.T) :
  if my_id != user_id :   # 나 말고 다른사람들과
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match :    # 코사인 유사도가 best_match보다 높을 경우
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

# 나와 가장 잘 맞는 user는 1150번이며, 코사인 유사도는 0.9999이다
print("Best Match : {}, Best Match ID : {}".format(best_match, best_match_id))

Best Match : 0.9999955006840267, Best Match ID : 1150


In [54]:
# 추천 목록 - 1150번 유저의 시청목록 중
recommend_list = []
for i, user_vector in enumerate(adj_matrix) :
  if adj_matrix[i][my_id] > 0.9 :
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 159, 160, 161, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 188, 189, 190, 192, 193, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 218, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 25

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [55]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [61]:
A, B, iter = non_negative_factorization(adj_matrix, n_components = 2)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [62]:
np.matmul(A, B)

array([[3.71108858e+00, 1.48454102e+00, 7.39535363e-01, ...,
        3.64490531e-03, 1.45506510e-02, 1.44110916e-02],
       [2.11724416e+00, 2.37338725e-01, 5.51650703e-01, ...,
        4.76092332e-03, 3.03585167e-05, 0.00000000e+00],
       [9.85272360e-01, 1.10447010e-01, 2.56713988e-01, ...,
        2.21552443e-03, 1.41275192e-05, 0.00000000e+00],
       ...,
       [1.04485667e+00, 1.17126289e-01, 2.72238757e-01, ...,
        2.34950819e-03, 1.49818804e-05, 0.00000000e+00],
       [1.45758857e+00, 5.42112787e-01, 2.99181695e-01, ...,
        1.61177659e-03, 5.15921394e-03, 5.10771005e-03],
       [2.44733103e+00, 9.41208708e-01, 4.95740038e-01, ...,
        2.56992125e-03, 9.08287324e-03, 8.99389282e-03]])

* 사용자 기반 추천

In [69]:
my_id, my_vector = 0, A[0]
best_match, best_match_id, best_match_vector = -1, -1, []   # 초기 유사도 설정

for user_id, user_vector in enumerate(A) :  # 비음수
  if my_id != user_id :   # 나 말고 다른사람들과
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match :    # 코사인 유사도가 best_match보다 높을 경우
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

# 나와 가장 잘 맞는 user는 314번이며, 코사인 유사도는 0.9999이다
print("Best Match : {}, Best Match ID : {}".format(best_match, best_match_id))

Best Match : 0.999999685629947, Best Match ID : 314


In [70]:
# 추천 목록 - 314번 유저의 시청목록
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])) :
  log1, log2 = log
  if log1 < 1. and log2 > 0. :
    recommend_list.append(i)
print(recommend_list)

[272, 275, 284, 285, 287, 300, 301, 302, 304, 317, 323, 326, 339, 381, 427, 430, 432, 460, 465, 474, 503, 507, 512, 519, 522, 530, 602, 641, 644, 650, 653, 656, 672, 708, 731, 740, 745, 769, 791, 1064, 1083]


- 항목 기반 추천

In [71]:
my_id, my_vector = 0, B.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []   # 초기 유사도 설정

for user_id, user_vector in enumerate(B.T) :
  if my_id != user_id :   # 나 말고 다른사람들과
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match :    # 코사인 유사도가 best_match보다 높을 경우
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

# 나와 가장 잘 맞는 user는 255번이며, 코사인 유사도는 0.9999이다
print("Best Match : {}, Best Match ID : {}".format(best_match, best_match_id))

Best Match : 0.9999999946284723, Best Match ID : 255


In [72]:
# 추천 목록 - 255번 유저의 시청목록 중
recommend_list = []
for i, user_vector in enumerate(adj_matrix) :
  if adj_matrix[i][my_id] > 0.9 :
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 